### SageMaker Counterfactual prediction

In [1]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)


In [2]:
version_tag="202304091951"
pytorch_custom_image_name=f"large-scale-ptm-ppi:gpu-{version_tag}"
instance_type = "ml.m5.2xlarge"  #ml.g4dn.2xlarge
instance_count = 1



In [3]:
docker_repo = "{}.dkr.ecr.{}.amazonaws.com/{}".format(account_id, region, pytorch_custom_image_name)

In [4]:
bucket = "aegovan-data"

In [5]:
import datetime
date_fmt = datetime.datetime.today().strftime("%Y%m%d%H")

In [6]:

s3_output_predictions = "s3://aegovan-data/counterfactuals/imdb/{}".format(date_fmt)



### Run  data prep

In [7]:
from sagemaker.network import NetworkConfig
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.processing import ScriptProcessor

script_processor = ScriptProcessor(image_uri=docker_repo,
                                       command=["python"],
                                       env={'mode': 'python', 'PYTHONPATH':'/opt/ml/code'},
                                       role=role,
                                       instance_type=instance_type,
                                       instance_count=instance_count,
                                       max_runtime_in_seconds= 5 * 24 * 60 * 60,
                                       volume_size_in_gb = 200,
                                       network_config=NetworkConfig(enable_network_isolation=False),
                                       base_job_name ="imdbdataprep"
                                       )


sm_local_input_models = "/opt/ml/processing/input/data/models"
sm_local_input_data = "/opt/ml/processing/input/data/jsondata"
sm_local_input_vocab = "/opt/ml/processing/input/data/vocab"


sm_local_output = "/opt/ml/processing/output"


script_processor.run(
        code='../src/utils/counterfactuals_imdb_dataprep.py',

        arguments=[
           
            "--outputdir", sm_local_output
        ],

      
        outputs=[ProcessingOutput(
                source=sm_local_output, 
                destination=s3_output_predictions,
                output_name='predictions')]
    )





Job Name:  imdbdataprep-2023-04-21-04-32-23-325
Inputs:  [{'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-2-324346001917/imdbdataprep-2023-04-21-04-32-23-325/input/code/counterfactuals_imdb_dataprep.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'predictions', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://aegovan-data/counterfactuals/imdb/2023042021', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
................{'outputdir': '/opt/ml/processing/output', 'log_level': 'INFO'}
2023-04-21 04:36:21,053 - __main__ - INFO - Train, val: ((15409, 3), (3853, 3))
2023-04-21 04:36:21,342 - __main__ - INFO - Counter factual train: (3414, 3)
2023-04-21 04:36:21,502 - __main__ - INFO - Counter factual val: (490, 3)
2023-04-21 04:36:21,961 - __main__ - INFO - Original train, val:

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2023-04-21 04:56:01,213 - __main__ - INFO - {"Unique": 1992, "PosRate": 0.5, "AdvRatePN": 0.291, "AffRateP": 0.008, "AffRateN": 0.004, "Total": 2000}
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2023-04-21 04:56:09,380 - __main__ - INFO - {"Unique": 400, "PosRate": 0.5, "AdvRatePN": 0.3, "AffRateP": 0.0, "AffRateN": 0.0, "Total": 400}
2023-04-21 04:56:09,389 - __main__ - INFO - Target Total size:2000,  target_neg_samples:1000 target_pos_samples: 1000
2023-04-21 04:56:09,391 - __main__ 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2023-04-21 05:07:50,189 - __main__ - INFO - {"Unique": 1996, "PosRate": 0.5, "AdvRatePN": 0.292, "AffRateP": 0.008, "AffRateN": 0.0, "Total": 2000}
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2023-04-21 05:07:58,072 - __main__ - INFO - {"Unique": 400, "PosRate": 0.5, "AdvRatePN": 0.295, "AffRateP": 0.0, "AffRateN": 0.0, "Total": 400}
2023-04-21 05:07:58,081 - __main__ - INFO - Target Total size:2000,  target_neg_samples:1000 target_pos_samples: 1000
2023-04-21 05:07:58,083 - __main__ 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2023-04-21 05:19:38,227 - __main__ - INFO - {"Unique": 1995, "PosRate": 0.5, "AdvRatePN": 0.288, "AffRateP": 0.008, "AffRateN": 0.0, "Total": 2000}
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2023-04-21 05:19:45,865 - __main__ - INFO - {"Unique": 400, "PosRate": 0.5, "AdvRatePN": 0.295, "AffRateP": 0.0, "AffRateN": 0.0, "Total": 400}
2023-04-21 05:19:45,874 - __main__ - INFO - Target Total size:2000,  target_neg_samples:1000 target_pos_samples: 1000
2023-04-21 05:19:45,876 - __main__ 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2023-04-21 05:31:32,265 - __main__ - INFO - {"Unique": 1994, "PosRate": 0.5, "AdvRatePN": 0.291, "AffRateP": 0.008, "AffRateN": 0.0, "Total": 2000}
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2023-04-21 05:31:40,581 - __main__ - INFO - {"Unique": 400, "PosRate": 0.5, "AdvRatePN": 0.3, "AffRateP": 0.0, "AffRateN": 0.0, "Total": 400}
2023-04-21 05:31:40,590 - __main__ - INFO - Target Total size:2000,  target_neg_samples:1000 target_pos_samples: 1000
2023-04-21 05:31:40,592 - __main__ -